In [11]:
import pandas as pd
import streamlit_authenticator as stauth

In [12]:
data = pd.read_excel('pegawai110.xlsx', dtype={'NIP':'str','KD':'str'},usecols=['NIP','Nama','Kantor','KD'])

In [13]:
data['password'] = data['NIP']+'@'+'Jaktim110'

In [14]:
data

,Nama,NIP,Kantor,KD,password
0,MUHAMMAD ISMIRANSYAH M. ZAIN,060077930,Kanwil DJP Jakarta Timur,110,060077930@Jaktim110
1,SUGENG SATOTO,060094070,Kanwil DJP Jakarta Timur,110,060094070@Jaktim110
2,DESSY EKA PUTRI,060087065,Kanwil DJP Jakarta Timur,110,060087065@Jaktim110
3,SANDRA BUANA,060083161,Kanwil DJP Jakarta Timur,110,060083161@Jaktim110
4,ARDHIE PERMADI,060083105,Kanwil DJP Jakarta Timur,110,060083105@Jaktim110
...,...,...,...,...,...
1082,NASTRIA PRATAMA,930302289,KPP Madya Dua Jakarta Timur,097,930302289@Jaktim110
1083,ADRY NOVIANTO,860013856,KPP Madya Dua Jakarta Timur,097,860013856@Jaktim110
1084,HERU IRFANTO,808320132,KPP Madya Dua Jakarta Timur,097,808320132@Jaktim110
1085,MIFTAKHUL PRATAMA PUTRA,810600567,KPP Madya Dua Jakarta Timur,097,810600567@Jaktim110


In [15]:
pass_hashed = stauth.Hasher(data['password'].tolist()).generate()

In [17]:
secrets = pd.DataFrame(pass_hashed, index=data.NIP.tolist(),columns=['PASS_HASHED'])

In [20]:
credential = pd.merge(data, secrets, left_on='NIP', right_index=True, how='inner')

In [21]:
credential

,Nama,NIP,Kantor,KD,password,PASS_HASHED
0,MUHAMMAD ISMIRANSYAH M. ZAIN,060077930,Kanwil DJP Jakarta Timur,110,060077930@Jaktim110,$2b$12$TXqPWPLGgXDy0hlQHwtBCu1D578eRjR9pKhWDAk...
1,SUGENG SATOTO,060094070,Kanwil DJP Jakarta Timur,110,060094070@Jaktim110,$2b$12$M9q6I6zd4bOPVexnHSvSU.g3ROlRPVNsFCyZRNp...
2,DESSY EKA PUTRI,060087065,Kanwil DJP Jakarta Timur,110,060087065@Jaktim110,$2b$12$rmV0tU0eViezIdEzA259dOuwYEJVwnPQJkVwxjd...
3,SANDRA BUANA,060083161,Kanwil DJP Jakarta Timur,110,060083161@Jaktim110,$2b$12$D/.gereEl4T..Y72GI4rm.rTRs9a6xRHcGiPhnf...
4,ARDHIE PERMADI,060083105,Kanwil DJP Jakarta Timur,110,060083105@Jaktim110,$2b$12$ryFdEuziF0EzMziPyI3VC.gyE5i.BS3XuDDdSKC...
...,...,...,...,...,...,...
1082,NASTRIA PRATAMA,930302289,KPP Madya Dua Jakarta Timur,097,930302289@Jaktim110,$2b$12$YGzPzHCcmbOBQwyM6BdPreJc1e2cIS1evOgUEha...
1083,ADRY NOVIANTO,860013856,KPP Madya Dua Jakarta Timur,097,860013856@Jaktim110,$2b$12$wdKWPEa1FYwCIwXR1al5sefSz/yPO8vpM6DIUuw...
1084,HERU IRFANTO,808320132,KPP Madya Dua Jakarta Timur,097,808320132@Jaktim110,$2b$12$OkFmwV4XvkIM5dO4wtoe7eAmwXiCilFuk4/pu8t...
1085,MIFTAKHUL PRATAMA PUTRA,810600567,KPP Madya Dua Jakarta Timur,097,810600567@Jaktim110,$2b$12$uqN/fCc5En/Jz0nAEgvUTeX0VCbOgaVf16Qq8AX...


In [22]:
import duckdb
db = duckdb.connect('credentials.duckdb')

In [23]:
db.sql('CREATE TABLE credential AS SELECT * FROM credential')

In [24]:
db.close()